In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from BFM.MCMC import Gibbs_sampling
from BFM.VI import NGVI

In [ ]:
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
P = 100
K = 15
N = 100



In [ ]:
B_0 = np.random.randn(P,K)
sigma2_0 = np.diag(np.ones(P))
cov_0 = B_0 @ B_0.T + sigma2_0


In [ ]:
X = np.random.multivariate_normal(np.zeros(P), cov_0, N)

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(cov_0)
plt.colorbar()
plt.title("Covariance")
plt.show()

In [ ]:
B_sample, sigma2_sample = Gibbs_sampling(torch.from_numpy(X).to(device1), a = 1, b = 1, c = 0.05, r = 30)

In [ ]:
mu, Precision, np_sigma = NGVI(torch.from_numpy(X).to(device1), a = 1, b = 100, c = 0.05, v = 1000, r = 30)

In [ ]:
A = torch.einsum('bij,bjk->bik',B_sample, B_sample.transpose(1,2))
for i in range(10000):
    A[i] = A[i] + torch.diag(sigma2_sample[i])

In [ ]:
v = 1000
B = mu @ mu.T + (v / (v-2)) * torch.diag(torch.vmap(torch.trace)(torch.linalg.inv(Precision))) + torch.diag(np_sigma / (0.5 * N))

In [ ]:
A_mean = A.mean(0)

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(A_mean)
plt.colorbar()
plt.title("Covariance")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(B.to('cpu'))
plt.colorbar()
plt.title("Covariance")
plt.show()

In [ ]:
cov_true = torch.from_numpy(cov_0)

In [ ]:
(cov_true - A_mean).abs().sum()/P**2

In [ ]:
(cov_true - A_mean).pow(2).sum()/P**2

In [ ]:
B_mean = B_sample.mean(0)

In [ ]:
for i in range(30):
    
    print(B_mean[:,i].abs().sum())

In [ ]:
(cov_true - B.to('cpu')).square().sum()/ P**2

In [ ]:
(cov_true - B.to('cpu')).abs().sum() / P**2

In [ ]:
for i in range(30):

    print(mu[:,i].abs().sum())